In [19]:
import csv
import re
import os
import pandas as pd
import numpy as np  

# Đọc file data.csv

In [20]:
df = pd.read_csv("data/data.csv", encoding="utf-8")
df.head()

,id,name,address,lat,lon,poi_type,avg_stars,total_reviews
0,0f9d2009-9436-46a4-b354-b0261898a39e,The Pub Coffee - Beer & Cocktail,"18A17 Tăng Nhơn Phú, Phước Long B, Quận 9, Thà...",10.829481,106.773785,"Cafe,Bar",4.9,181
1,02887955-963a-43ac-b0f7-355d7d7cfacf,Julieta,"C. Sta. Lucía, 9, Distrito Centro, 29008 Málag...",36.722011,-4.421780,Cafe,4.3,2053
2,622c7643-30e8-4402-9b6c-b8407ff063e2,BAIETA Restaurant Saigon Thao Dien,"Nguyễn Văn Hưởng, Thảo Điền, 16/8, Thành phố H...",10.802978,106.727268,"Restaurant,Bar,Cocktail bar,Coffee shop,Family...",4.5,285
3,4f06908d-e9fa-4f6a-b1ae-c7d8882e2edf,Parroquia San Felipe Neri,"C. Guerrero, 6, Distrito Centro, 29012 Málaga,...",36.725554,-4.421321,"Catholic church,Tourist attraction",4.7,329
4,279dfce3-c227-4b58-b4ed-09197327a32a,BÀ BAR,"15 Đ. Nguyễn Cừ, Thảo Điền, Quận 2, Thành phố ...",10.803190,106.728381,"Cocktail bar,Cafe",4.8,182


# Bước 1: Tìm giá trị min-max của avg_stars

In [21]:
min_avg = df["avg_stars"].min()
max_avg = df["avg_stars"].max()
print(f"avg_stars: min={min_avg}, max={max_avg}")

avg_stars: min=3.0, max=5.0


# Bước 2: Tìm giá trị min-max của total_reviews

In [22]:
min_total_reviews = df["total_reviews"].min()
max_total_reviews = df["total_reviews"].max()
print(f"total_reviews: min={min_total_reviews}, max={max_total_reviews}")

total_reviews: min=1, max=76447


# Bước 3: Chuẩn hóa avg_stars bằng Min-Max Scaling

Công thức: `(x - min) / (max - min)`

Kết quả: giá trị trong khoảng [0, 1]

In [23]:
df["avg_stars_norm"] = ((df["avg_stars"] - min_avg) / (max_avg - min_avg)).round(3)
print("Mẫu avg_stars_norm:")
print(df[["avg_stars", "avg_stars_norm"]].head(10))

Mẫu avg_stars_norm:
   avg_stars  avg_stars_norm
0        4.9            0.95
1        4.3            0.65
2        4.5            0.75
3        4.7            0.85
4        4.8            0.90
5        4.9            0.95
6        4.7            0.85
7        5.0            1.00
8        4.3            0.65
9        4.0            0.50


# Bước 4: Chuẩn hóa total_reviews bằng Log Transform

Công thức: `log(x + 1) / log(max + 1)`

Lý do dùng log: total_reviews có phạm vi rất rộng (1 đến hàng chục nghìn), log transform giúp giảm độ chênh lệch và phân phối đều hơn.

In [24]:
df["total_reviews_norm"] = (np.log(df["total_reviews"] + 1) / np.log(max_total_reviews + 1)).round(3)
print("Mẫu total_reviews_norm:")
print(df[["total_reviews", "total_reviews_norm"]].head(10))

Mẫu total_reviews_norm:
   total_reviews  total_reviews_norm
0            181               0.463
1           2053               0.678
2            285               0.503
3            329               0.516
4            182               0.463
5            165               0.455
6            216               0.478
7            275               0.500
8           1322               0.639
9           1321               0.639


# Bước 5: Tính điểm tổng hợp normalize_stars_reviews

Công thức: `(avg_stars_norm + total_reviews_norm) / 2`

Kết quả: Trung bình cộng của 2 chỉ số đã chuẩn hóa, cho điểm đánh giá tổng hợp từ [0, 1]

In [25]:
df["normalize_stars_reviews"] = ((df["avg_stars_norm"] + df["total_reviews_norm"]) / 2).round(3)
print("Mẫu kết quả cuối cùng:")
print(df[["name", "avg_stars", "total_reviews", "avg_stars_norm", "total_reviews_norm", "normalize_stars_reviews"]].head(10))

Mẫu kết quả cuối cùng:
                                                name  avg_stars  \
0                   The Pub Coffee - Beer & Cocktail        4.9   
1                                            Julieta        4.3   
2                 BAIETA Restaurant Saigon Thao Dien        4.5   
3                          Parroquia San Felipe Neri        4.7   
4                                             BÀ BAR        4.8   
5                                   Art Bar Viet Nam        4.9   
6  Ngỡ Cafe | Cà phê view đẹp Thủ Đức | Coffee ng...        4.7   
7                             TEEMAY COFFEE ROASTERS        5.0   
8                       La Mafia se sienta a la mesa        4.3   
9                                         Bar tocata        4.0   

   total_reviews  avg_stars_norm  total_reviews_norm  normalize_stars_reviews  
0            181            0.95               0.463                    0.706  
1           2053            0.65               0.678                    0.664  

# Bước 6: Kiểm tra kết quả

In [26]:
# Kiểm tra công thức có đúng không
print("Kiểm tra công thức với 3 hàng đầu tiên:\n")
for i in range(3):
    avg_norm = df.iloc[i]["avg_stars_norm"]
    rev_norm = df.iloc[i]["total_reviews_norm"]
    result = df.iloc[i]["normalize_stars_reviews"]
    calculated = round((avg_norm + rev_norm) / 2, 3)
    
    print(f"Hàng {i+1}: {df.iloc[i]['name'][:50]}")
    print(f"  avg_stars_norm = {avg_norm}")
    print(f"  total_reviews_norm = {rev_norm}")
    print(f"  normalize_stars_reviews = {result}")
    print(f"  Tính lại: ({avg_norm} + {rev_norm}) / 2 = {calculated}")
    print(f"  ✅ Đúng!" if result == calculated else f"  ❌ SAI!")
    print()

print(f"\nThống kê:")
print(f"  avg_stars_norm: min={df['avg_stars_norm'].min()}, max={df['avg_stars_norm'].max()}")
print(f"  total_reviews_norm: min={df['total_reviews_norm'].min()}, max={df['total_reviews_norm'].max()}")
print(f"  normalize_stars_reviews: min={df['normalize_stars_reviews'].min()}, max={df['normalize_stars_reviews'].max()}")

Kiểm tra công thức với 3 hàng đầu tiên:

Hàng 1: The Pub Coffee - Beer & Cocktail
  avg_stars_norm = 0.95
  total_reviews_norm = 0.463
  normalize_stars_reviews = 0.706
  Tính lại: (0.95 + 0.463) / 2 = 0.706
  ✅ Đúng!

Hàng 2: Julieta
  avg_stars_norm = 0.65
  total_reviews_norm = 0.678
  normalize_stars_reviews = 0.664
  Tính lại: (0.65 + 0.678) / 2 = 0.664
  ✅ Đúng!

Hàng 3: BAIETA Restaurant Saigon Thao Dien
  avg_stars_norm = 0.75
  total_reviews_norm = 0.503
  normalize_stars_reviews = 0.626
  Tính lại: (0.75 + 0.503) / 2 = 0.626
  ✅ Đúng!


Thống kê:
  avg_stars_norm: min=0.0, max=1.0
  total_reviews_norm: min=0.062, max=1.0
  normalize_stars_reviews: min=0.031, max=0.924


# Bước 7: Xóa các cột trung gian (giữ lại dữ liệu gốc và kết quả cuối)

In [27]:
df = df.drop(columns=["avg_stars_norm", "total_reviews_norm"])
print("Cột còn lại trong DataFrame:")
print(df.columns.tolist())
print("\nMẫu dữ liệu cuối cùng:")
df.head()

Cột còn lại trong DataFrame:
['id', 'name', 'address', 'lat', 'lon', 'poi_type', 'avg_stars', 'total_reviews', 'normalize_stars_reviews']

Mẫu dữ liệu cuối cùng:


,id,name,address,lat,lon,poi_type,avg_stars,total_reviews,normalize_stars_reviews
0,0f9d2009-9436-46a4-b354-b0261898a39e,The Pub Coffee - Beer & Cocktail,"18A17 Tăng Nhơn Phú, Phước Long B, Quận 9, Thà...",10.829481,106.773785,"Cafe,Bar",4.9,181,0.706
1,02887955-963a-43ac-b0f7-355d7d7cfacf,Julieta,"C. Sta. Lucía, 9, Distrito Centro, 29008 Málag...",36.722011,-4.421780,Cafe,4.3,2053,0.664
2,622c7643-30e8-4402-9b6c-b8407ff063e2,BAIETA Restaurant Saigon Thao Dien,"Nguyễn Văn Hưởng, Thảo Điền, 16/8, Thành phố H...",10.802978,106.727268,"Restaurant,Bar,Cocktail bar,Coffee shop,Family...",4.5,285,0.626
3,4f06908d-e9fa-4f6a-b1ae-c7d8882e2edf,Parroquia San Felipe Neri,"C. Guerrero, 6, Distrito Centro, 29012 Málaga,...",36.725554,-4.421321,"Catholic church,Tourist attraction",4.7,329,0.683
4,279dfce3-c227-4b58-b4ed-09197327a32a,BÀ BAR,"15 Đ. Nguyễn Cừ, Thảo Điền, Quận 2, Thành phố ...",10.803190,106.728381,"Cocktail bar,Cafe",4.8,182,0.682


In [29]:
df["normalize_stars_reviews"].max()

np.float64(0.924)

# Bước 8: Xuất dữ liệu ra file CSV

In [28]:
df.to_csv("data/data_clean_normalize.csv", index=False, encoding="utf-8")
print("✅ Đã xuất dữ liệu thành công vào: data/data_clean_normalize.csv")
print(f"Tổng số dòng: {len(df)}")
print(f"Tổng số cột: {len(df.columns)}")

✅ Đã xuất dữ liệu thành công vào: data/data_clean_normalize.csv
Tổng số dòng: 1454
Tổng số cột: 9
